In [1]:
import pickle
import pandas as pd
import numpy as np
new_df = pickle.load(open('new.pkl','rb'))
similarity = pickle.load(open('similarity.pkl','rb'))
cleaned_df = pd.read_csv('cleaned_df.csv')

Search

In [2]:
# import json
# import urllib.request as urllib_request
# import ssl

# def fetch_poster_path(api_key, base_url, movie_id):
#     poster_url = f"{base_url}{movie_id}?api_key={api_key}&language=en-US"
#     print(poster_url)
#     response = urllib_request.urlopen(poster_url)
#     json_data = json.loads(response.read())

#     # Extract the poster path from the API response
#     poster_path = json_data.get('poster_path', '')
    
#     return f"https://image.tmdb.org/t/p/w500/{poster_path}"


In [3]:
import urllib.request as urllib_request
import ssl
import json
import time

def fetch_poster_path(api_key, base_url, movie_id):
    # Construct the poster URL
    poster_url = f"{base_url}{movie_id}?api_key={api_key}&language=en-US"
    
    try:
        print("before Fetching")
        # Open the URL with a timeout of 0.3 seconds
        start_time = time.time()
        print("just after time")
        response = urllib_request.urlopen(poster_url,timeout=1)
        end_time = time.time()
        print("After timeout")
        elapsed_time = end_time - start_time
        
        # Check if the response was successful and the elapsed time is within the limit
        if response.getcode() == 200 and elapsed_time < 1:
            # Extract the poster path from the API response
            json_data = json.loads(response.read())
            poster_path = json_data.get('poster_path', '')
            print("before return statement")
            return f"https://image.tmdb.org/t/p/w500/{poster_path}"
    except Exception as e:
        # Handle other exceptions
        print(f"An error occurred: {e}")
        return None

    # Return a default picture path if fetching takes longer than 0.3 seconds or encounters an error
    return None

In [4]:
def fetch_movie_details(api_key, base_url, movie_id):
    details_url = f"{base_url}{movie_id}?api_key={api_key}&language=en-US"
    print("issue in overview")
    try:
        response = urllib_request.urlopen(details_url)
        details_data = json.loads(response.read())
        overview = details_data.get('overview', '')
    except Exception as e:  
        print(f"An error occurred: {e}")
        details_data = "No review here"
        overview = details_data
        
    # Extract the movie overview from the API response
    
    print("sup")
    return overview

In [5]:
def recommend_20(movie):
    movie_list = []

    # Check if movie is None or an empty string
    if movie is None or not movie.strip():
        print("Error: Movie is None or empty.")
        return movie_list

    # Normalize movie title to lowercase and remove leading/trailing spaces
    movie = movie.strip().lower()

    # Check if movie exists in DataFrame
    if movie in new_df['title'].str.lower().values:
        index = new_df[new_df['title'].str.lower() == movie].index[0]
    else:
        # Movie not found in DataFrame
        print(f"Movie '{movie}' not found in DataFrame.")
        return movie_list

    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])

    for i in distances[0:20]:
        movie_index = i[0]
        movie_title = new_df.iloc[movie_index]['title']

        # Fetch the poster path dynamically from the TMDB API
        # api_key = "ff1e18b29203a4007ed0df1907cccf71"
        api_key = "aecbf4dee6af30c2f4f736f275694ab3"
        
        base_url = "https://api.themoviedb.org/3/movie/"
        movie_id = new_df.iloc[movie_index]['movie_id']

        poster_path = fetch_poster_path(api_key, base_url, movie_id)
        overview = fetch_movie_details(api_key, base_url, movie_id)

        movie_list.append({
            'title': movie_title,
            'poster_path': poster_path,
            'overview': overview,
            'movie_id': movie_id
        })

    return movie_list

Collaborative Filtering

In [6]:
df_ratings = pd.read_csv('rate.csv')

In [7]:
from scipy.sparse import csr_matrix

def create_X(df):
    """
    Generates a sparse matrix from ratings dataframe.

    Args:
        df: pandas dataframe containing 3 columns (userId, movieId, rating)

    Returns:
        X: sparse matrix
        user_mapper: dict that maps user id's to user indices
        user_inv_mapper: dict that maps user indices to user id's
        movie_mapper: dict that maps movie id's to movie indices
        movie_inv_mapper: dict that maps movie indices to movie id's
    """
    M = df['userId'].nunique()
    N = df['movie_id'].nunique()
    user_mapper = dict(zip(np.unique(df["userId"]), list(range(M))))
    movie_mapper = dict(zip(np.unique(df["movie_id"]), list(range(N))))

    user_inv_mapper = dict(zip(list(range(M)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(N)), np.unique(df["movie_id"])))

    user_index = [user_mapper[i] for i in df['userId']]
    item_index = [movie_mapper[i] for i in df['movie_id']]

    X = csr_matrix((df["rating"], (user_index,item_index)), shape=(M,N))

    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_X(df_ratings)

In [8]:
from sklearn.neighbors import NearestNeighbors

def knn_similar_users(userId, X, user_mapper, user_inv_mapper, k, metric='cosine'):
    """
    Finds k-nearest neighbours for a given movie id.

    Args:
        movie_id: id of the movie of interest
        X: user-item utility matrix
        k: number of similar movies to retrieve
        metric: distance metric for kNN calculations

    Output: returns list of k similar movie ID's
    """
    neighbour_ids = []

    user_index = user_mapper[userId]
    user_vector = X[user_index]
    if isinstance(user_vector, (np.ndarray)):
        user_vector = user_vector.reshape(1,-1)
    # use k+1 since kNN output includes the movieId of interest
    kNN = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric=metric)
    kNN.fit(X)
    neighbour = kNN.kneighbors(user_vector, return_distance=False)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(user_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids

In [9]:
def recommended_movies_by_collab_filtering(userId):
    movie_list = []
    similar_users = knn_similar_users(userId,X,user_mapper,user_inv_mapper,k=10)
    df1 = df_ratings.loc[df_ratings['userId'] == userId].sort_values(by='rating', ascending = False)
    df2 = df_ratings.loc[df_ratings['userId'] == similar_users[0]].sort_values(by='rating', ascending = False)
    movies_notseen = list(set(df2['movie_id']) - set(df1['movie_id']))
    filtered_df = df_ratings[(df_ratings['userId'] == similar_users[0]) & (df_ratings['movie_id'].isin(movies_notseen))].sort_values(by='rating',ascending=False)
    movies = filtered_df.iloc[0:20,1:2]
    rec_movies = movies['movie_id'].tolist()
    for i in rec_movies:
        movie_id = i
        movie_title = new_df.loc[new_df['movie_id'] == i, 'title'].iloc[0]
        
        # Fetch the poster path dynamically from the TMDB API
        # api_key = "ff1e18b29203a4007ed0df1907cccf71"
        api_key = "aecbf4dee6af30c2f4f736f275694ab3"
        base_url = "https://api.themoviedb.org/3/movie/"
    
        poster_path = fetch_poster_path(api_key, base_url, movie_id)
        overview = fetch_movie_details(api_key, base_url, movie_id)
        
        movie_list.append({
            'title': movie_title,
            'poster_path': poster_path,
            'overview': overview,
            'movie_id': movie_id
        })

    return movie_list

Recommend by Genre

In [10]:
movies_genres = pd.read_csv('tmdb_5000_movies.csv')
movies_genres = movies_genres.rename(columns={"id":"movie_id"})
movies_genres = movies_genres[['movie_id','title','genres','vote_average']]

In [11]:
import ast

def convert(text):            #helper function to fetch appropriate element from genres and keywords
    lst = []
    for i in ast.literal_eval(text):        #converts string of list to list
        lst.append(i['name'])
    return lst

movies_genres['genres'] = movies_genres['genres'].apply(convert)

In [12]:
def filter_movies_by_genres(df, genres):
    """
    Filter movies DataFrame based on genres entered by the user.
    
    Args:
        df (DataFrame): DataFrame containing movies and genres.
        genres (list): List of genres entered by the user.
        
    Returns:
        DataFrame: Filtered DataFrame containing movies matching the genres.
    """
    filtered_df = df[df['genres'].apply(lambda x: all(genre in x for genre in genres))]
    return filtered_df

In [13]:
def genre_filterer_by_fans(genres):
    movie_list = []
    filtered_movies = filter_movies_by_genres(movies_genres, genres)
    sorted_frame = filtered_movies.sort_values(by='vote_average',ascending=False)
    final = sorted_frame.iloc[0:20,0:1]
    rec_movies = final['movie_id'].tolist()
    
    for i in rec_movies:
        movie_id = i
        movie_title = new_df.loc[new_df['movie_id']==i,'title'].iloc[0]

        # Fetch the poster path dynamically from the TMDB API
        # api_key = "ff1e18b29203a4007ed0df1907cccf71"
        api_key = "aecbf4dee6af30c2f4f736f275694ab3"
        base_url = "https://api.themoviedb.org/3/movie/"

        poster_path = fetch_poster_path(api_key, base_url, movie_id)
        overview = fetch_movie_details(api_key, base_url, movie_id)
        
        movie_list.append({
            'title': movie_title,
            'poster_path': poster_path,
            'overview': overview,
            'movie_id': movie_id
        })

    return movie_list

In [14]:
old_userId = 0
movieList = []
Movies_by_genre = []
def check_session(userId,genres):
    global old_userId,movieList,Movies_by_genre
    if old_userId == userId:
        print("In current session\nNo updation needed")
    else:
        print("updating")
        movieList = recommended_movies_by_collab_filtering(userId)
        Movies_by_genre = genre_filterer_by_fans(genres)
    old_userId = userId
    return movieList,Movies_by_genre

In [15]:
from flask import Flask
from flask_mysqldb import MySQL
from flask import render_template, redirect, request, url_for, flash, session,request as flask_request
import hashlib
import json
from jinja2 import Environment

app = Flask(__name__)

app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = '121418'
app.config['MYSQL_DB'] = 'movie_rec'

app.config['MYSQL_CURSORCLASS'] = 'DictCursor'

# api_key = "ff1e18b29203a4007ed0df1907cccf71"
api_key = "aecbf4dee6af30c2f4f736f275694ab3"
base_url = "https://api.themoviedb.org/3/discover/movie?api_key=" + api_key

mysql = MySQL(app)

app.secret_key = 'your_secret_key'


def jsonify(obj):
    return json.dumps(obj)

app.jinja_env.filters['jsonify'] = jsonify

# Hashing function
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        name = request.form['name']
        genres = request.form.getlist('genre')
        try:
            cur = mysql.connection.cursor()
            cur.execute("INSERT INTO users (keyname, username, pass, genres) VALUES (%s, %s, %s, %s)",
                        (name, username, password, ','.join(genres)))
            mysql.connection.commit()
            cur.close()

            flash('Registration successful. You can now login.', 'success')
            return redirect(url_for('login'))
        except Exception as e:
            flash('Registration failed. Please try again later.', 'danger')
            print(f"An error occurred during registration: {e}")
            
    return render_template('register.html')


@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        # password = hashlib.sha256(request.form['password'].encode()).hexdigest()
        password = request.form['password']
        print(username)
        print(password)

        try:
            cur = mysql.connection.cursor()
            cur.execute("SELECT * FROM users WHERE username = %s AND pass = %s", (username, password))
            user = cur.fetchone()
            cur.close()

            if user:
                session['username'] = user['username']
                session['user_id'] = user['id']
                session['genres'] = user['genres']
                genres = list(session['genres'].split(','))
                print("login toh hua")
                check_session(session['user_id'],genres)
                flash('Login successful', 'success')
                return redirect(url_for('home'))
            else:
                flash('Invalid credentials', 'danger')
        except Exception as e:
            flash('Login failed. Please try again later.', 'danger')
            print(f"An error occurred during login: {e}")
    return render_template('login.html')


@app.route('/logout')
def logout():
    session.clear()
    flash('You have been logged out.', 'info')
    return redirect(url_for('login'))

@app.route('/')
def index():
    return redirect(url_for('login'))

@app.route('/home', methods=['GET', 'POST'])
def home():
    
    if flask_request.method == 'POST':
        search_query = flask_request.form.get('search')
        print(f"Search Query: {search_query}")
        recommended_movies = recommend_20(search_query)
        print(f"Recommended Movies: {recommended_movies}")
        return render_template("home.html", recommended_movies=recommended_movies, search_query=search_query)
    genres = list(session['genres'].split(','))
    movieList,Movies_by_genre = check_session(session['user_id'],genres)
    print(movieList)
    return render_template("home.html", data=movieList,movies = Movies_by_genre,search_query=None)

@app.route('/discover', methods=['GET', 'POST'])
def discover():
    
    if flask_request.method == 'POST':
        search_query = flask_request.form.get('search')
        print(f"Search Query: {search_query}")

        recommended_movies = recommend_20(search_query)
        print(f"Recommended Movies: {recommended_movies}")
        return render_template("index.html", recommended_movies=recommended_movies, search_query=search_query)

    ssl._create_default_https_context = ssl._create_unverified_context
    # Set the number of movies you want to display
    num_movies_to_display = 50
    
    # Calculate the number of pages required to display the specified number of movies
    num_pages = (num_movies_to_display + 19) // 20

    # Fetch the data for each page and concatenate the results
    all_results = []
    for page in range(1, num_pages + 1):
        page_url = base_url + f"&page={page}"
        conn = urllib_request.urlopen(page_url)
        json_data = json.loads(conn.read())
        all_results.extend(json_data["results"])
    return render_template("index.html", data=all_results, search_query=None)

@app.route("/movie_details/<int:movie_id>")
def movie_details(movie_id):
    # Get details from the DataFrame based on movie_id
    movie_details_data = cleaned_df[cleaned_df['movie_id'] == movie_id].iloc[0]


    # api_key = "ff1e18b29203a4007ed0df1907cccf71"
    api_key = "aecbf4dee6af30c2f4f736f275694ab3"
    base_url = "https://api.themoviedb.org/3/movie/"
    
    # Extract relevant information from movie_details_data
    movie_details = {
        'title': movie_details_data['title'],
        'poster_path': fetch_poster_path(api_key,base_url,movie_id),
        'genres': movie_details_data['genres'],
        'cast': movie_details_data['cast'],
        'crew': movie_details_data['crew'],
        'overview': movie_details_data['overview'],
        'movie_id' : movie_details_data['movie_id']
    }

    return render_template("movie_details.html", movie_details=movie_details)

@app.route("/review/<int:movie_id>")
def review(movie_id):
    # api_key = "ff1e18b29203a4007ed0df1907cccf71"
    api_key = "aecbf4dee6af30c2f4f736f275694ab3"
    base_url = "https://api.themoviedb.org/3/movie/"
    
    movie_details_data = cleaned_df[cleaned_df['movie_id'] == movie_id].iloc[0]
    movie_details = {
        'title': movie_details_data['title'],
        'poster_path': fetch_poster_path(api_key, base_url, movie_id),
        'movie_id' : movie_id
    }
    return render_template("review.html",movie_details=movie_details)

@app.route('/submit_rating', methods=['POST'])
def submit_rating():
    data = request.json
    rating = data['rating']
    review = data['review']
    movieId = data['movieId']
    print(data)
    try:
        cur = mysql.connection.cursor()
        cur.execute("INSERT INTO reviews (stars, movieId, review) VALUES (%s, %s, %s)",
                    (rating, movieId,review))
        mysql.connection.commit()
    except Exception as e:
        print(f"found: {e}")
        if 'Duplicate entry' in str(e):
            try:
                cur.execute("UPDATE reviews SET stars = %s, review = %s WHERE movieId = %s",
                            (rating, review, movieId))
                mysql.connection.commit()
                print("Done updating")
            except Exception as e:
                print(f"Error occurred during update: {e}")
    finally:
        cur.close()
    return jsonify({'success': True})

@app.before_request
def require_login():
    allowed_routes = ['login', 'register','logout']  # add other allowed routes
    if 'user_id' not in session and request.endpoint not in allowed_routes:
        return redirect(url_for('login'))
    
if __name__ == "__main__":
    app.run()
    
    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Apr/2024 08:38:05] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [13/Apr/2024 08:38:06] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2024 08:38:08] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2024 08:38:25] "POST /register HTTP/1.1" 302 -
127.0.0.1 - - [13/Apr/2024 08:38:25] "GET /login HTTP/1.1" 200 -


new
asd
login toh hua
updating
before Fetching
just after time
After timeout
before return statement
issue in overview
sup
before Fetching
just after time
After timeout
before return statement
issue in overview
sup
before Fetching
just after time
After timeout
before return statement
issue in overview
sup
before Fetching
just after time
After timeout
before return statement
issue in overview
sup
before Fetching
just after time
After timeout
before return statement
issue in overview
sup
before Fetching
just after time
After timeout
before return statement
issue in overview
sup
before Fetching
just after time
After timeout
before return statement
issue in overview
sup
before Fetching
just after time
After timeout
before return statement
issue in overview
sup
before Fetching
just after time
After timeout
before return statement
issue in overview
sup
before Fetching
just after time
After timeout
before return statement
issue in overview
sup
before Fetching
just after time
After timeout
bef

127.0.0.1 - - [13/Apr/2024 08:38:54] "

After timeout
before return statement
issue in overview
sup


...

After timeout
before return statement
before Fetching
just after time


127.0.0.1 - - [13/Apr/2024 08:40:25] "GET /review/601 HTTP/1.1" 200 -


After timeout
before return statement


127.0.0.1 - - [13/Apr/2024 08:40:31] "POST /submit_rating HTTP/1.1" 200 -


{'rating': 3, 'review': 'good', 'movieId': '601'}
found: (1146, "Table 'movie_rec.reviews' doesn't exist")
